In [ ]:
import os
import pandas as pd
from nba_api.stats.static import teams
from pathlib import Path

# Setup data directory path - go up one level from notebooks to project root
BASE_DIR = Path(os.getcwd()).parent
DATA_DIR = BASE_DIR / 'data' / 'game6'
os.makedirs(DATA_DIR, exist_ok=True)

print(f"Working directory: {BASE_DIR}")
print(f"Data directory: {DATA_DIR}")
print(f"Data directory exists: {DATA_DIR.exists()}")
print(f"Data directory is writable: {os.access(DATA_DIR, os.W_OK)}")

# Get team IDs
OKC_data = teams.find_teams_by_full_name("Oklahoma City Thunder")
OKC_id = OKC_data[0]['id'] # 1610612760

PAC_data = teams.find_teams_by_full_name("Indiana Pacers")
PAC_id = PAC_data[0]['id'] # 1610612754


In [ ]:
# Get Game ID
from nba_api.stats.endpoints import leaguegamefinder

finder = leaguegamefinder.LeagueGameFinder(
    player_or_team_abbreviation="T",
    team_id_nullable=OKC_id,
    date_from_nullable="06/19/2025",
    date_to_nullable="06/19/2025"
)

g6 = finder.get_data_frames()[0]
g6_id = g6["GAME_ID"].iloc[0] # Should be 0042400406

print("Game Data:")
print(g6)


In [ ]:
# Import required endpoints
from nba_api.stats.endpoints import (
    boxscoretraditionalv2,
    boxscoreadvancedv2,
    playbyplayv2,
    hustlestatsboxscore,
    winprobabilitypbp,
    boxscorefourfactorsv2,
    shotchartdetail
)

# Traditional Stats
trad_stats = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=g6_id)
trad_stats_df = trad_stats.get_data_frames()[0]
trad_stats_df.to_csv(DATA_DIR / 'traditional_stats.csv', index=False)

# Advanced Stats
adv_stats = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=g6_id)
adv_stats_df = adv_stats.get_data_frames()[0]
adv_stats_df.to_csv(DATA_DIR / 'advanced_stats.csv', index=False)

# Play by Play
pbp = playbyplayv2.PlayByPlayV2(game_id=g6_id)
pbp_df = pbp.get_data_frames()[0]
pbp_df.to_csv(DATA_DIR / 'play_by_play.csv', index=False)

# Hustle Stats
hustle = hustlestatsboxscore.HustleStatsBoxScore(game_id=g6_id)
hustle_df = hustle.get_data_frames()[0]
hustle_df.to_csv(DATA_DIR / 'hustle_stats.csv', index=False)

# Win Probability
wp = winprobabilitypbp.WinProbabilityPBP(game_id=g6_id)
wp_df = wp.get_data_frames()[0]
wp_df.to_csv(DATA_DIR / 'win_probability.csv', index=False)

# Four Factors
ff = boxscorefourfactorsv2.BoxScoreFourFactorsV2(game_id=g6_id)
ff_df = ff.get_data_frames()[0]
ff_df.to_csv(DATA_DIR / 'four_factors.csv', index=False)

# Shot Charts
okc_shots = shotchartdetail.ShotChartDetail(team_id=OKC_id, game_id_nullable=g6_id)
okc_shots_df = okc_shots.get_data_frames()[0]
okc_shots_df.to_csv(DATA_DIR / 'okc_shots.csv', index=False)

pac_shots = shotchartdetail.ShotChartDetail(team_id=PAC_id, game_id_nullable=g6_id)
pac_shots_df = pac_shots.get_data_frames()[0]
pac_shots_df.to_csv(DATA_DIR / 'pac_shots.csv', index=False)

print("All data has been collected and saved to the data/game6 directory.")
